# Web Scrapping causal #

In [1]:
import selenium
from requests import get
from bs4 import BeautifulSoup 
from selenium import webdriver
from datetime import datetime
from re import sub
import pandas as pd

In [2]:
url_dict = {2010:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2009-stars",
            2011:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2010-stars",
            2012:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2011-stars",
            2013:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2012-stars",
           2014:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2013-stars",
           2015:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2014-stars",
           2016:"https://www.the-numbers.com/box-office-star-records/worldwide/yearly-acting/highest-grossing-2015-stars"}
actors_data = {}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
for year,url in url_dict.items():
    actors_data[year] = set() 
    response = get(url, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find("tbody").find_all("a")
    for actor in container:
        actors_data[year].add(actor.find(text=True))

In [3]:
def get_date(tag):
    try:
        str_date = tag.find('td').text
        date = datetime.strptime(str_date,"%b %d, %Y")
        return date
    except:
        return None
def get_world_wide_box_office(tag):
    try:
        return int(sub(r'[^\d.]', '', tag.find_all('td')[-1].text))
    except:
        return None

In [4]:
production_url = "https://www.the-numbers.com/movies/production-companies/"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
}
response = get(production_url,headers=headers)
html_soup = BeautifulSoup(response.text, 'html.parser')
container = html_soup.find("tbody").find_all("a")
counter = 0
studios_data = []
for x in container:
    print(x['href'])
    counter+=1
    response = get("https://www.the-numbers.com"+x['href'],headers=headers)
    html_soup_movie = BeautifulSoup(response.text, 'html.parser')
    movies_of_company = [t for t in html_soup_movie.find("tbody") if t.find('td') != -1]
    records_for_company = [(get_date(t),get_world_wide_box_office(t)) for t in movies_of_company]
    records_for_company = [(date,income) for (date,income) in records_for_company if date is not None and income is not None]
    studio_data = pd.DataFrame.from_records(records_for_company, columns =['Date', 'Income'])
    studio_data['company'] = x['href'].split('/')[-1]
    studios_data.append(studio_data)
    if counter > 250:
        break


/movies/production-company/Columbia-Pictures
/movies/production-company/Warner-Bros
/movies/production-company/Universal-Pictures
/movies/production-company/Walt-Disney-Pictures
/movies/production-company/Paramount-Pictures
/movies/production-company/Marvel-Studios
/movies/production-company/20th-Century-Fox
/movies/production-company/New-Line-Cinema
/movies/production-company/Relativity-Media
/movies/production-company/DreamWorks-Pictures
/movies/production-company/Legendary-Pictures
/movies/production-company/Amblin-Entertainment
/movies/production-company/Dune-Entertainment
/movies/production-company/Pixar
/movies/production-company/Regency-Enterprises
/movies/production-company/Village-Roadshow-Productions
/movies/production-company/DreamWorks-Animation
/movies/production-company/Lucasfilm
/movies/production-company/Metro-Goldwyn-Mayer-Pictures
/movies/production-company/Lionsgate
/movies/production-company/Touchstone-Pictures
/movies/production-company/RatPac-Entertainment
/movies

/movies/production-company/Robert-Simonds-Company
/movies/production-company/Team-Todd
/movies/production-company/Closest-to-the-Hole
/movies/production-company/3-Arts-Entertainment
/movies/production-company/DC-Films
/movies/production-company/Mark-Gordon-Company-The
/movies/production-company/Phoenix-Pictures
/movies/production-company/Nu-Image
/movies/production-company/Saturn-Films
/movies/production-company/Marv-Films
/movies/production-company/Good-Universe
/movies/production-company/Di-Novi-Pictures
/movies/production-company/UK-Film-Council
/movies/production-company/Cross-Creek-Pictures
/movies/production-company/Maverick-Films
/movies/production-company/Sony-Pictures
/movies/production-company/Beacon-Pictures
/movies/production-company/Laurence-Mark
/movies/production-company/Rogue-Pictures
/movies/production-company/Mark-Johnson
/movies/production-company/Stage-6-Films
/movies/production-company/Valhalla-Motion-Pictures
/movies/production-company/CBS-Films
/movies/production

In [5]:
bins = [[datetime(year-5,1,1),datetime(year,1,1)] for year in range(2012,2016)]    

In [6]:
companies_data= {}
for data in studios_data:
    company_name = data['company'].unique()[0]
    companies_data[company_name] = {}
    for bin_ in bins:
        data_temp = data[(data['Date']>=bin_[0]) & (data['Date']<bin_[1])]
        companies_data[company_name][bin_[1].year]=data_temp['Income'].mean()

In [7]:
complete_data = pd.DataFrame.from_dict(companies_data,orient='index')
complete_data= complete_data.stack().reset_index()
complete_data= complete_data.rename({'level_1':'Year','level_0':'Company',0:'5 years average'},axis=1)
complete_data

,Company,Year,5 years average
0,Columbia-Pictures,2012,1.922935e+08
1,Columbia-Pictures,2013,2.153690e+08
2,Columbia-Pictures,2014,2.222168e+08
3,Columbia-Pictures,2015,2.355555e+08
4,Warner-Bros,2012,2.278504e+08
...,...,...,...
815,87Eleven-Inc,2015,5.875286e+07
816,Trigger-Street-Productions,2012,5.439744e+07
817,Trigger-Street-Productions,2013,5.117188e+07
818,Trigger-Street-Productions,2014,6.637232e+07


In [61]:
directors_urls = ["https://www.imdb.com/list/ls056848274/","https://www.imdb.com/list/ls056848274/?sort=list_order,asc&mode=detail&page=2",
       "https://www.imdb.com/list/ls056848274/?sort=list_order,asc&mode=detail&page=3"]
top_directors = []
for url in directors_urls:
    response = get(url,headers=headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find("div",class_="lister-list").find_all("div",class_="lister-item mode-detail")
    for director in container:
        director= director.find("h3",class_="lister-item-header").text.split('.')[1].strip()
        top_directors.append(director)
        

In [62]:
top_directors

['Steven Spielberg',
 'Martin Scorsese',
 'Alfred Hitchcock',
 'Stanley Kubrick',
 'Quentin Tarantino',
 'Orson Welles',
 'Francis Ford Coppola',
 'Ridley Scott',
 'Akira Kurosawa',
 'Joel Coen',
 'Ethan Coen',
 'John Ford',
 'Sergio Leone',
 'Woody Allen',
 'Billy Wilder',
 'Clint Eastwood',
 'Frank Capra',
 'Charles Chaplin',
 'Christopher Nolan',
 'Roman Polanski',
 'Ingmar Bergman',
 'David Lynch',
 'Fritz Lang',
 'David Fincher',
 'Peter Jackson',
 'Federico Fellini',
 'Tim Burton',
 'James Cameron',
 'Sidney Lumet',
 'Brian De Palma',
 'François Truffaut',
 'Terrence Malick',
 'David Cronenberg',
 'William Wyler',
 'Robert Zemeckis',
 'Ang Lee',
 'Howard Hawks',
 'George Lucas',
 'Mel Brooks',
 'Terry Gilliam',
 'Milos Forman',
 'Steven Soderbergh',
 'John Huston',
 'Werner Herzog',
 'Oliver Stone',
 'David Lean',
 'Danny Boyle',
 'Rob Reiner',
 'Buster Keaton',
 'Michael Curtiz',
 'Sam Raimi',
 'Michael Mann',
 'Robert Altman',
 'Sam Peckinpah',
 'Peter Weir',
 'Spike Lee',
 'Je

In [63]:
scrapped_data = {'Jerry Bruckheimer Films', 'Village Roadshow Productions', 'Plan B Entertainment', 'Cruel and Unusual Films', 'Newmarket Films', 'Eon Productions', 'Anonymous Content', 'Dark Castle Entertainment', 'M6 Films', 'Miramax Films', 'Metro Goldwyn Mayer Pictures', 'Lionsgate', 'Laurence Mark', 'Mace Neufeld', 'Warner Animation Group', 'Sunswept Entertainment', 'Regency Enterprises', 'CBS Films', 'Lin Pictures', 'Mandeville Films', 'United Artists', 'Arad Productions', 'Color Force', 'Kennedy Marshall Company The', 'Brian Grazer Productions', 'Constantin Film', 'Di Novi Pictures', 'Heyday Films', 'Rainforest Films', 'Band Apart A', 'Revolution Studios', 'Point Grey', 'Nu Image', 'Phoenix Pictures', 'Everyman Pictures', 'Judd Apatow', 'Big Screen Prods', 'Lucasfilm', 'Tapestry Films', 'Trigger Street Productions', 'UK Film Council', 'Overbrook Entertainment', 'Scott Rudin Productions', 'Bluegrass Films', 'Sony Pictures', 'Original Film', 'Image Nation Abu Dhabi', 'New Regency', 'Jersey Films', 'Hartbeat Productions', 'Gary Sanchez Productions', 'Robert Simonds Company', 'Bagdasarian Company', 'Will Packer Productions', 'di Bonaventura Pictures', 'Waverly Films', 'Cube Vision', '3 Arts Entertainment', '20th Century Fox', 'Millennium Films', 'Bad Hat Harry Productions', 'Chernin Entertainment', 'Valhalla Motion Pictures', 'Imagine Entertainment', 'Cruise Wagner', 'Fuji Television Network', 'Media Rights Capital', 'Jerry Bruckheimer', 'Jerry Weintraub', 'Gil Netter Productions', 'Alcon Entertainment', 'Touchstone Pictures', 'Castle Rock Entertainment', 'Cine Plus', 'Tig Productions', 'Participant Media', 'Appian Way', 'RatPac Entertainment', 'Tribeca Film', 'Fox 2000 Pictures', 'Icon Productions', 'Walt Disney Animation Studios', 'Paramount Pictures', 'Entertainment One', 'Mad Chance', 'Hemisphere Media Capital', 'Leverage', 'Birnbaum Barber', 'Laura Ziskin Productions', 'Tri Star Pictures', 'Gold Circle Films', 'Don Murphy', 'Bad Robot', 'Sony Pictures Animation', 'EuropaCorp', 'Wigram Productions', 'Junction Entertainment', 'GK Films', 'Flynn Picture Company', 'Hasbro Studios', 'Sidney Kimmel Entertainment', 'De Line Pictures', 'Green Hat Films', 'Nickelodeon Films', 'Dune Entertainment', 'Kerner Entertainment Company', 'Tom DeSanto', 'Solana Films', 'Screen Gems', 'Morgan Creek Productions', 'Red Hour Productions', 'Jon Peters', 'Stage 6 Films', 'Canal Plus', 'Closest to the Hole', 'Tom Cruise Productions', 'Focus Features', 'LStar Capital', 'Troublemaker Studios', 'Twentieth Century Fox', 'Chris Meledandri', 'Hollywood Pictures', 'Image Nation', 'BBC Films', 'Pixar', 'Dentsu Inc', 'Mandate Pictures', 'Beacon Pictures', 'Grive Productions', 'Syncopy', 'Red Wagon Entertainment', 'Escape Artists', 'DreamWorks Animation', 'Ingenious Media', 'Roth Films', 'Fox Searchlight Pictures', 'Imprint Entertainment', 'Alphaville', 'DreamWorks Pictures', '1492 Pictures', 'New Line Cinema', 'Silver Pictures', 'Parkes Plus Macdonald Productions', 'FilmNation Entertainment', 'Cross Creek Pictures', 'Mosaic Media', 'American Zoetrope', 'Spyglass Entertainment', '87Eleven Inc', 'Smart Entertainment', 'Reliance Entertainment', 'Legendary Pictures', 'MTV Films', 'Benderspink Productions', 'PDI', 'Atlas Entertainment', 'MRC', 'Open Road Films', 'One Race Films', 'Annapurna Pictures', 'Dimension Films', 'Thunder Road Film', 'Temple Hill Entertainment', 'Weed Road Pictures', 'Blumhouse', 'Malpaso Productions', 'Michael De Luca Productions', 'ImageMovers', 'Working Title Films', 'Reliance BIG Entertainment', 'Tyler Perry Studios', 'Mark Gordon Company The', 'Mark Johnson', 'Blinding Edge Pictures', 'Montecito Picture Company', 'Illumination Entertainment', 'Platinum Dunes', 'Centropolis Entertainment', 'Rogue Pictures', 'Mutual Film Company', 'Ghost House Pictures', 'Blue Sky Studios', 'DMG Entertainment', 'Walt Disney Pictures', 'Safran Company The', 'Scott Free Films', '20th Century Fox Animation', 'Wingnut Films', 'Columbia Pictures', 'Olive Bridge Entertainment', 'Warner Bros', 'Gracie Films', 'Davis Entertainment', 'Animal Logic', 'Vertigo Entertainment', 'Lawrence Gordon', '21 Laps Entertainment', 'Marv Films', 'Ingenious Film Partners', 'Wendy Fineman Productions', 'Lakeshore Entertainment', 'Summit Entertainment', 'Contrafilm', 'Radar Pictures', 'Zanuck Company', 'Happy Madison', 'Team Todd', 'Marc Platt Productions', 'Kinberg Genre', 'StudioCanal', 'Walden Media', 'Good Universe', 'TSG Entertainment', 'Broken Road', 'Marvel Studios', 'Universal Pictures', 'Donners Company', 'Playtone', 'Weinstein Company', 'Relativity Media', 'Maverick Films', 'Saturn Films', 'Fairview Entertainment', 'Atomic Monster', 'Matt Tolmach Productions', 'Skydance Productions', 'Amblin Entertainment', 'Pariah Productions'}
all_data={'Pure Flix Entertainment', 'A24 Films', 'Double Feature Films', 'Arc Entertainment', 'Lionsgate', 'Paramount Studios', 'CBS Films', 'Mandeville Films', 'Indomina Releasing', 'Dreamworks Pictures', 'Envision Media Arts', 'Ombra Films', 'Image Entertainment', 'Film 44', 'Freestyle Releasing', 'Sony Pictures Columbia', 'Samuel Goldwyn Films', 'Sony Pictures', 'Screwed Film', 'Attack Entertainment', 'Will Packer Productions', 'Oscilloscope Pictures', '20th Century Fox', 'Lionsgate Films', 'Millenium Entertainment', 'Well Go USA', 'Sony Films', 'Spitfire Studios', 'First Run Features', 'Touchstone Pictures', 'Participant Media', 'Anchor Bay Entertainment', 'A24 and DIRECTV', 'Entertainment One', 'Paramount Pictures', 'IFC Films', 'Rocky Mountain Pictures', 'Underhill Films', 'Anchor Bay', 'Alchemy', 'Vertical Entertainment', 'EuropaCorp', 'Aviron', 'TriStar Pictures', 'Eone Films', 'The Film Arcade', 'Brainstorm Media', 'Film District', 'Universal Studios', 'Overture Films', 'Starburns Industries', 'Focus Features', 'Twentieth Century Fox', 'Indepedent Pictures', 'Magnolia Releasing', 'Xlrator Media', 'STX Entertainment', 'IFC Midnight', 'Beat Pirate Motion Pictures', 'Sony Pictures Classics', 'HBO Films', 'Bow and Arrow Entertainment', 'James Patterson Entertainment', 'Mike and Marty Productions', 'Blackbird Films', 'Open Road Films', 'Independent Pictures', 'Naedomi Media', 'Keep Your Head', 'Rooster Teeth Productions', 'Sony Screen Gems', 'FilmBuff', 'Fuego Films', 'Benaroya Pictures', 'The Weinstein Co.', 'Oscilloscope Laboratories', 'Focus World', 'Magnolia Pictures', 'DreamWorks Studios', 'Strand Releasing', 'IFC', 'Fox Searchlight', 'The Weinstein Company', 'Entertainment One Films', 'Gener8Xion Entertainment', 'Walt Disney Pictures', 'Warner Bros. Pictures', 'Element Pictures', 'Traverse Media', 'Columbia Pictures', 'Paladin', 'ESX Productions', 'Warner Bros.', 'Pure Flix', 'Best Served Cold Productions', 'The Asylum', 'Summit Entertainment', 'Screen Media Films', 'MGM', 'Broad Green Pictures', 'Strategic Films', 'Picturehouse Films', 'Phase 4 Films', 'Ten Furlongs LLC.', 'Relativity Studios', 'LD Entertainment', 'Pantelion Films', 'Night and Day Pictures', 'A24', 'Universal Pictures', 'Relativity Media', 'Radius-TWC', 'Xenon Pictures', 'Roadside Attractions', 'Big Beach Films', 'Fox Searchlight Pictures'}


In [68]:
all_data.difference(scrapped_data)

{'A24',
 'A24 Films',
 'A24 and DIRECTV',
 'Alchemy',
 'Anchor Bay',
 'Anchor Bay Entertainment',
 'Arc Entertainment',
 'Attack Entertainment',
 'Aviron',
 'Beat Pirate Motion Pictures',
 'Benaroya Pictures',
 'Best Served Cold Productions',
 'Big Beach Films',
 'Blackbird Films',
 'Bow and Arrow Entertainment',
 'Brainstorm Media',
 'Broad Green Pictures',
 'Double Feature Films',
 'DreamWorks Studios',
 'Dreamworks Pictures',
 'ESX Productions',
 'Element Pictures',
 'Entertainment One Films',
 'Envision Media Arts',
 'Eone Films',
 'Film 44',
 'Film District',
 'FilmBuff',
 'First Run Features',
 'Focus World',
 'Fox Searchlight',
 'Freestyle Releasing',
 'Fuego Films',
 'Gener8Xion Entertainment',
 'HBO Films',
 'IFC',
 'IFC Films',
 'IFC Midnight',
 'Image Entertainment',
 'Indepedent Pictures',
 'Independent Pictures',
 'Indomina Releasing',
 'James Patterson Entertainment',
 'Keep Your Head',
 'LD Entertainment',
 'Lionsgate Films',
 'MGM',
 'Magnolia Pictures',
 'Magnolia Rele

In [72]:
sorted(scrapped_data)

['1492 Pictures',
 '20th Century Fox',
 '20th Century Fox Animation',
 '21 Laps Entertainment',
 '3 Arts Entertainment',
 '87Eleven Inc',
 'Alcon Entertainment',
 'Alphaville',
 'Amblin Entertainment',
 'American Zoetrope',
 'Animal Logic',
 'Annapurna Pictures',
 'Anonymous Content',
 'Appian Way',
 'Arad Productions',
 'Atlas Entertainment',
 'Atomic Monster',
 'BBC Films',
 'Bad Hat Harry Productions',
 'Bad Robot',
 'Bagdasarian Company',
 'Band Apart A',
 'Beacon Pictures',
 'Benderspink Productions',
 'Big Screen Prods',
 'Birnbaum Barber',
 'Blinding Edge Pictures',
 'Blue Sky Studios',
 'Bluegrass Films',
 'Blumhouse',
 'Brian Grazer Productions',
 'Broken Road',
 'CBS Films',
 'Canal Plus',
 'Castle Rock Entertainment',
 'Centropolis Entertainment',
 'Chernin Entertainment',
 'Chris Meledandri',
 'Cine Plus',
 'Closest to the Hole',
 'Color Force',
 'Columbia Pictures',
 'Constantin Film',
 'Contrafilm',
 'Cross Creek Pictures',
 'Cruel and Unusual Films',
 'Cruise Wagner',
 '